# X3FL dynamic relationships

## The relationship

In [1]:
import numpy as np
import pandas as pd

In [2]:
all_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran", "Yaki"]
# https://imgur.com/nqI0nbO, by blazenclaw, forum.egosoft.com
try:
    relationship_df = pd.read_csv("relationship.csv", index_col=0)
except:
    relationship_df = pd.read_csv("https://raw.githubusercontent.com/mkmark/X3FL-dynamic-relationships/main/relationship.csv", index_col=0)
relationship_df = pd.DataFrame(relationship_df, columns=all_races)
relationship_df

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki
Teladi,1.00,0.15,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,-0.3
NMMC,0.15,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.30,0.00,0.00,0.0,0.0
Goner,0.00,0.00,1.00,0.15,0.00,0.05,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
TerraCorp,0.00,0.00,0.15,1.00,-0.15,0.15,0.05,-0.15,0.05,-0.30,0.00,-0.30,-0.30,0.0,-0.3
Strong Arms,0.00,0.00,-0.30,-0.15,1.00,-0.15,-0.30,0.15,-0.30,-0.15,0.00,0.05,0.00,-0.3,0.0
Argon,0.00,0.00,0.05,0.15,-0.15,1.00,0.15,-0.15,0.05,0.15,-0.30,-0.30,-0.30,-0.3,-0.3
Boron,0.00,0.00,0.05,0.05,-0.30,0.15,1.00,-0.30,0.15,0.05,-0.30,-0.15,-0.30,-0.3,-0.3
Split,0.00,0.00,0.00,-0.15,0.15,-0.15,-0.30,1.00,-0.30,-0.15,-0.30,0.15,0.00,-0.3,-0.3
Arteus,0.00,0.00,0.00,0.05,-0.30,0.05,0.15,-0.30,1.00,-0.30,0.00,-0.15,-0.30,-0.3,-0.3
OTAS,0.00,0.00,0.00,-0.30,-0.15,0.15,0.05,-0.15,-0.30,1.00,0.00,-0.30,-0.30,-0.3,-0.3


The problem is simple: given the relationship $\mathbf{R}$, target notority $\mathbf{N}$, find tactics $\mathbf{X}$, which is a vector that represents how much notority point one should get from a race.

$$
\mathbf{R}\mathbf{X}=\mathbf{N}
$$

$$
\mathbf{X}=\mathbf{R}^{-1}\mathbf{N}
$$

Define $\mathbf{N}$ as vectors of 1, since we don't want to lose effort by hitting the notoriety ceiling.

In [3]:
def get_X(races):
    R = np.array(relationship_df.loc[races, races])
    invR = np.linalg.inv(R) 
    N = np.ones(len(races))
    X = invR@N
    return X

def get_X_df(races):
    X = get_X(races)
    return pd.DataFrame(list(X)+[sum(X)]+[len(races)/sum(X)], index=races+['sum', 'efficiency']).transpose()

## Example solutions

Let's see if there is a solution for all the races

The number for each race represents how much effort you will have to put with that race to get 1 notoriety point for each race assuming you're to be friend with all these selected race.

The sum is the total workload of current tactic.

The efficiency is the sum of actual gained notoriety points (equal the number of races selected) divided by total workload.

In [4]:
get_X_df(all_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,Yaki,sum,efficiency
0,0.367565,0.900265,1.185814,-0.86887,0.134097,-0.55656,-0.553109,-0.753074,-1.073404,-1.401387,-0.148668,-1.436434,-1.380733,-1.648176,-1.657983,-8.890657,-1.687164


Unfortunately there is not a solution as one would have to get negative points from races. The problem is, when actively losing points of a race, the notoriety points of enemy races does not increase accordingly. However if that is true, we can see a path that you do some missions for some races but keep robbing other races and one day you will find yourself master of diplomacy with all races - too good to be true for those races you robbed.

Let's get results of some interesting races combo.

In [5]:
commonwealth_races = ["Teladi", "Argon", "Boron", "Split", "Paranid"]
major_races = ["Teladi", "Argon", "Boron", "Split", "Paranid", "Terran"]
all_but_yaki_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates", "Terran"]
all_but_yaki_terran_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Pirates"]
all_but_yaki_terran_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid"]
all_but_yaki_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Terran"]
all_but_terran_pirates_races = ["Teladi", "NMMC", "Goner", "TerraCorp", "Strong Arms", "Argon", "Boron", "Split", "Arteus", "OTAS", "Duke's", "Paranid", "Yaki"]

All commonwealth races, no problem

In [6]:
get_X_df(commonwealth_races)

,Teladi,Argon,Boron,Split,Paranid,sum,efficiency
0,1.0,1.428571,1.428571,1.428571,1.428571,6.714286,0.744681


All major races, no problem

In [7]:
get_X_df(major_races)

,Teladi,Argon,Boron,Split,Paranid,Terran,sum,efficiency
0,1.0,3.823529,3.823529,3.823529,3.823529,5.588235,21.882353,0.274194


All but Yaki, not an option

In [8]:
get_X_df(all_but_yaki_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,Pirates,Terran,sum,efficiency
0,0.895285,0.698097,1.28077,-1.190619,-0.728985,-0.965686,-1.077861,-1.113347,-1.825664,-2.202037,-0.5587,-2.251337,-2.253901,-2.893255,-14.187239,-0.986802


According to calculation just 3 enemies seems not sufficient

In [9]:
get_X_df(all_but_yaki_terran_pirates_races)

,Teladi,NMMC,Goner,TerraCorp,Strong Arms,Argon,Boron,Split,Arteus,OTAS,Duke's,Paranid,sum,efficiency
0,-0.223804,8.158693,-2.784266,16.185269,12.932356,11.639467,15.490037,19.493104,16.696293,20.359163,23.750408,23.837658,165.53438,0.072492


## Let's explore all possibilities

In [10]:
# https://stackoverflow.com/questions/26332412/python-recursive-function-to-display-all-subsets-of-given-set
def subs(l):
    if l == []:
        return [[]]

    x = subs(l[1:])

    return x + [[l[0]] + y for y in x]

Turned out there are the following number of possible combos.

In [11]:
least_enemy_set_list = []
for races in subs(all_races):
    if races != []:
        if min(get_X(races))>0:
            enemy_set = set(all_races) - set(races)
            is_duplicate = False
            for i, least_enemy_set in enumerate(least_enemy_set_list):
                if enemy_set.issubset(least_enemy_set):
                    least_enemy_set_list[i] = enemy_set
                    is_duplicate = True
                    break
                if least_enemy_set.issubset(enemy_set):
                    is_duplicate = True
                    break
            if not is_duplicate:
                least_enemy_set_list = least_enemy_set_list + [enemy_set]
                
len(least_enemy_set_list)

4322

which is the following (hided)

In [12]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    workload = sum(get_X(list(set(all_races) - least_enemy_set)))
    efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
    item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
    #print(item)
    parsed_least_enemy_set_list += [item]

3 or less enemies option are listed below (none)

In [13]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) <= 3:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

4 enemies

In [14]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 4:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Pirates - Yaki - Paranid - Terran, 38.148199989746054, 0.28834912270976654
TerraCorp - Yaki - Paranid - Terran, 71.65090538050065, 0.1535221354368759
Pirates - TerraCorp - Yaki - Terran, 76.66451155984795, 0.1434822941696156


5 enemies but without Terran

In [15]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 5 and "Terran" not in least_enemy_set:
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Split - Pirates - Strong Arms - Yaki - Paranid, 33.82163512524274, 0.29566873283830414


6 enemies but without any major races

In [16]:
parsed_least_enemy_set_list = []
for least_enemy_set in least_enemy_set_list:
    if len(list(least_enemy_set)) == 6 and set(major_races).intersection(least_enemy_set) == set():
        workload = sum(get_X(list(set(all_races) - least_enemy_set)))
        efficiency = (len(all_races) - len(list(least_enemy_set)))/workload
        item = " - ".join(list(least_enemy_set)) + ", " + str(workload) + ", " + str(efficiency)
        print(item)
        parsed_least_enemy_set_list += [item]

Duke's - Pirates - Arteus - TerraCorp - Yaki - Goner, 310.8756766195213, 0.02895047981195081
Duke's - Pirates - OTAS - Strong Arms - Yaki - Goner, 54.95335871443448, 0.16377524887547937
Duke's - Pirates - OTAS - Arteus - Yaki - Goner, 60.530840393152324, 0.14868453736218312
Duke's - Pirates - Arteus - Strong Arms - Yaki - Goner, 109.36868104097832, 0.08229046848089788
Duke's - Pirates - OTAS - Arteus - Strong Arms - Yaki, 27.08258851233183, 0.3323168313804984
Duke's - Pirates - Arteus - Strong Arms - TerraCorp - Yaki, 46.59709570869033, 0.19314508475517508
Duke's - Pirates - OTAS - Strong Arms - TerraCorp - Yaki, 46.59709570869033, 0.19314508475517508
Duke's - Pirates - OTAS - Arteus - TerraCorp - Yaki, 47.00593896196852, 0.19146516799253185


## Conclusion

- A simple tool is developed
- All possible combos are given with total workload and efficiency
- Strong Arms - OTAS - Duke's - Arteus - Pirates - Yaki is one of the least enemy approaches to keep good relationship with all 6 major races, and is the most efficient way among